<strong> Capstone Class Notebook </strong>

Sources:
Neighborhood population->*Age: https://www.arcgis.com/home/webmap/viewer.html?webmap=344e36b89430403f814963fe207f0017-tools/neighborhood-statistics-demographics.page

Rent prices->https://www.loopnet.com/new-york/new-york_restaurants-for-lease/2/

In [1]:
#All required libraries will be imported here
import numpy as np
import pandas as pd
import xlrd

In [2]:
#Enter the name of the uploaded spreadsheet.
loc = "Dataset_finalproject.xlsx"

In [3]:
#Convert the spreadsheet into a dataframe
df_pop_age = pd.read_excel(loc)
df_pop_age.head()

,Neighborhood,Population,Adults under 30,Average Rent,Lat,Long
0,Battery Park/Lower Manhattan,41461,10085,21000,40.7033,-74.0170
1,East Harlem,114093,20019,4800,40.7957,-73.9389
2,Upper East Side,102735,15529,28000,40.7736,-73.9566
3,SoHo/Tribeca,96953,18795,8000,40.7233,-74.0030
4,Gramercy/Murray Hill,183061,48472,7000,40.7479,-73.9757


In [4]:
#Add a column for the percentage of adults under 30, and calculate the percentages
df_pop_age['Percent of Adults under 30'] = df_pop_age["Adults under 30"]/df_pop_age["Population"]*100

#Create a new database, and reorder the columns. Round the Lat and Long, and drop rows without a neighborhood
df_with_averages = df_pop_age[['Neighborhood', 'Percent of Adults under 30', 'Average Rent', 'Lat', 'Long']]
df_with_averages = df_with_averages.round(2)
df_with_averages = df_with_averages.dropna(subset=['Neighborhood'])
df_with_averages.head()


,Neighborhood,Percent of Adults under 30,Average Rent,Lat,Long
0,Battery Park/Lower Manhattan,24.32,21000,40.70,-74.02
1,East Harlem,17.55,4800,40.80,-73.94
2,Upper East Side,15.12,28000,40.77,-73.96
3,SoHo/Tribeca,19.39,8000,40.72,-74.00
4,Gramercy/Murray Hill,26.48,7000,40.75,-73.98


The above dataframe is the base dataframe that the restaurant data will be added to. Next we have to create the restaurant data.

In [5]:
#Import and download other libraries that will be needed later.
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests

    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

In [6]:
#Input foursquare credentials
CLIENT_ID = 'OVWKIVZTKAQK3GTX2PG0XO4ESUCBBYEYKT4Q4UW40SO5FI33'
CLIENT_SECRET = 'CWHXW2D20OMRC5H115UQL1CEOBYTCHECZV5PPIQJLTJDJINO'
VERSION = '20180604'
LIMIT = 30

In [7]:
#Attempt to iterate through each neighborhood in the dataframe, and find how many Mexican restaurants are within 1 km of the center of each neighborhood.
search_query = 'Mexican'
radius = 1000 #in meters
num_restaurants = [] #create an empty list to keep track of the number of restaurants in each neighborhood

for index, col in df_with_averages.iterrows():
    latitude = df_with_averages['Lat'][index] #latitude of each neighborhood
    longitude = df_with_averages['Long'][index] #longitude of each neighborhood
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    df_restaurants = json_normalize(venues)
    count = len(df_restaurants.index)
    num_restaurants.append(count)

print(num_restaurants)

[5, 20, 12, 22, 30, 22, 30, 3, 7, 30, 8, 17, 10, 24, 6, 17, 9]


In [8]:
#Create a dictionary for the number of restaurants
num_dict = {'Num of Restaurants':num_restaurants}

In [9]:
#Convert the number of restaurants dictionary into a dataframe
num_rest_df = pd.DataFrame.from_dict(num_dict)

In [10]:
#Merge the dataframes, drop the Lat and Long columns
frames = [df_with_averages, num_rest_df]
final_df = pd.concat(frames, axis = 1, join='outer', sort = False)

final_df = final_df[['Neighborhood', 'Percent of Adults under 30', 'Average Rent', 'Num of Restaurants']]
final_df.head()

,Neighborhood,Percent of Adults under 30,Average Rent,Num of Restaurants
0,Battery Park/Lower Manhattan,24.32,21000,5
1,East Harlem,17.55,4800,20
2,Upper East Side,15.12,28000,12
3,SoHo/Tribeca,19.39,8000,22
4,Gramercy/Murray Hill,26.48,7000,30


In [11]:
#Divide the 'Average Rent' and 'Num of Restaurants' by the minimum value so we can easily see the variances.
min_rent = final_df['Average Rent'].min()
final_df['Average Rent'] = final_df['Average Rent']/min_rent

min_rest = final_df['Num of Restaurants'].min()
final_df['Num of Restaurants'] = final_df['Num of Restaurants']/min_rest

final_df['Adults under 30'] = final_df['Percent of Adults under 30']/final_df['Percent of Adults under 30'].max()

final_df['Neighborhood Score'] = final_df['Average Rent']*final_df['Num of Restaurants']/final_df['Adults under 30']


final_df = final_df.drop(columns=['Percent of Adults under 30'])


final_df.head()

,Neighborhood,Average Rent,Num of Restaurants,Adults under 30,Neighborhood Score
0,Battery Park/Lower Manhattan,16.153846,1.666667,0.918429,29.314271
1,East Harlem,3.692308,6.666667,0.662764,37.140478
2,Upper East Side,21.538462,4.000000,0.570997,150.883191
3,SoHo/Tribeca,6.153846,7.333333,0.732251,61.629442
4,Gramercy/Murray Hill,5.384615,10.000000,1.000000,53.846154


Note: The average rent, and number of restaurant, columns were normalized with the lowest value considered the best. The Adults under 30 column with the largest values are normalized to the highest value, and a value of 1 is considered the best. For all 3 columns used to determine the "Neighborhood Score" the ideal value is one. This means that we are looking for a "Neighborhood Score" of 1.

In [12]:
final_df = final_df[['Neighborhood', 'Adults under 30', 'Average Rent', 'Num of Restaurants', 'Neighborhood Score']]

final_df.head(18)

,Neighborhood,Adults under 30,Average Rent,Num of Restaurants,Neighborhood Score
0,Battery Park/Lower Manhattan,0.918429,16.153846,1.666667,29.314271
1,East Harlem,0.662764,3.692308,6.666667,37.140478
2,Upper East Side,0.570997,21.538462,4.000000,150.883191
3,SoHo/Tribeca,0.732251,6.153846,7.333333,61.629442
4,Gramercy/Murray Hill,1.000000,5.384615,10.000000,53.846154
5,Chinatown,0.724698,6.000000,7.333333,60.714956
6,West Village,0.973943,13.769231,10.000000,141.376204
7,Harlem,0.621979,3.330000,1.000000,5.353880
8,Upper West Side,0.657855,26.153846,2.333333,92.764579
9,Midtown,0.853097,11.769231,10.000000,137.958933
